In [54]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [55]:
#!pip install scikit-learn

In [56]:
with open('intents.json') as file:
    data = json.load(file)

In [57]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [58]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [60]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [65]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [66]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_9 (Dense)             (None, 16)                272       
                                                                 
 dense_10 (Dense)            (None, 16)                272       
                                                                 
 dense_11 (Dense)            (None, 10)                170       
                                                                 
Total params: 16,714
Trainable params: 16,714
Non-trainable params: 0
__________________________________________________

In [67]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
2/2 [==============================] - 0s 2ms/step - loss: 2.3015 - accuracy: 0.0513
Epoch 2/550
2/2 [==============================] - 0s 1ms/step - loss: 2.2996 - accuracy: 0.1026
Epoch 3/550
2/2 [==============================] - 0s 1ms/step - loss: 2.2986 - accuracy: 0.1282
Epoch 4/550
2/2 [==============================] - 0s 1ms/step - loss: 2.2975 - accuracy: 0.1282
Epoch 5/550
2/2 [==============================] - 0s 1ms/step - loss: 2.2967 - accuracy: 0.1282
Epoch 6/550
2/2 [==============================] - 0s 2ms/step - loss: 2.2957 - accuracy: 0.1282
Epoch 7/550
2/2 [==============================] - 0s 2ms/step - loss: 2.2947 - accuracy: 0.1282
Epoch 8/550
2/2 [==============================] - 0s 1ms/step - loss: 2.2938 - accuracy: 0.1282
Epoch 9/550
2/2 [==============================] - 0s 1ms/step - loss: 2.2928 - accuracy: 0.1282
Epoch 10/550
2/2 [==============================] - 0s 1ms/step - loss: 2.2916 - accuracy: 0.1282
Epoch 11/550
2/2 [===========

In [68]:
# saving model
model.save("chat_model.h5")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)